In [1]:
import pickle

heterodata = pickle.load(open("hetero_graph_final.pkl", "rb"))

c:\Users\I549512\AppData\Local\miniconda3\envs\gnnpapersearch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
heterodata

HeteroData(
  paper={
    license=[1000],
    doi=[1000],
    pages=[1000],
    journal=[1000],
    date=[1000],
    id=[1000],
    name=[1000],
    num_nodes=1000,
  },
  author={
    name=[2886],
    num_nodes=2886,
  },
  category={
    name=[108],
    num_nodes=108,
  },
  word={
    name=[10667],
    num_nodes=10667,
  },
  journal={
    name=[574],
    num_nodes=574,
  },
  (paper, written_by, author)={ edge_index=[2, 3068] },
  (paper, has_category, category)={ edge_index=[2, 1502] },
  (paper, has_word, word)={
    edge_index=[2, 77703],
    edge_attr=[77703],
  },
  (paper, has_titleword, word)={ edge_index=[2, 7462] },
  (paper, in_journal, journal)={ edge_index=[2, 1000] },
  (word, co_occurs_with, word)={
    edge_index=[2, 499132],
    edge_attr=[499132],
  }
)

In [3]:
heterodata['paper']['license'] = [license if license is not None else 'None' for license in heterodata['paper']['license']]

In [4]:

# remove isolated nodes
import torch_geometric.transforms as T
transform = T.Compose([
       #T.RemoveIsolatedNodes(),
       T.RemoveDuplicatedEdges(),
       # T.ToUndirected(merge=False) # don't merge reversed edges into the original edge type
])
heterodata = transform(heterodata)

In [5]:
import torch
# id mapping so we can remove the attributes and then remove isolated nodes, later we can add the attributes back
id_dict = {}
more_mappings = {}
heterodata_dict = heterodata.to_dict()
for nodetype in heterodata.node_types:
    print(nodetype)
    ids = torch.arange(heterodata[nodetype].num_nodes) 
    id_mapping = {i.item():name for i, name in zip(ids, heterodata[nodetype].name)}
    if nodetype =='paper':
        for key in ['license','doi','pages','journal','date','id']:
            more_mappings[key] = {i.item():name for i, name in zip(ids, heterodata[nodetype][key])}
    
    heterodata[nodetype].x = ids.squeeze(-1)
    id_dict[nodetype] = id_mapping
    
    for key in heterodata_dict[nodetype].keys():
        if key != 'x':
            print('del', nodetype, key)
            del heterodata[nodetype][key]

paper
del paper license
del paper doi
del paper pages
del paper journal
del paper date
del paper id
del paper name
del paper num_nodes
author
del author name
del author num_nodes
category
del category name
del category num_nodes
word
del word name
del word num_nodes
journal
del journal name
del journal num_nodes


In [6]:
# check correct edge types:
transform = T.Compose([
       T.RemoveIsolatedNodes(),
       #    T.RemoveDuplicatedEdges(),
       # T.ToUndirected(merge=False) # don't merge reversed edges into the original edge type
])

heterodata = transform(heterodata)

In [7]:
heterodata


HeteroData(
  paper={ x=[1000] },
  author={ x=[2886] },
  category={ x=[108] },
  word={ x=[10604] },
  journal={ x=[574] },
  (paper, written_by, author)={ edge_index=[2, 3067] },
  (paper, has_category, category)={ edge_index=[2, 1502] },
  (paper, has_word, word)={
    edge_index=[2, 57073],
    edge_attr=[57073],
  },
  (paper, has_titleword, word)={ edge_index=[2, 7354] },
  (paper, in_journal, journal)={ edge_index=[2, 1000] },
  (word, co_occurs_with, word)={
    edge_index=[2, 499132],
    edge_attr=[499132],
  }
)

In [8]:
heterodata = T.ToUndirected()(heterodata)


In [9]:
# map back
for nodetype in heterodata.node_types:
    print(nodetype)
    id_mapping = id_dict[nodetype]
    heterodata[nodetype].name = [id_mapping[i.item()] for i in heterodata[nodetype].x]
    
    heterodata[nodetype].num_nodes = len(heterodata[nodetype].name)
    
    if nodetype =='paper':
        for key in ['license','doi','pages','journal','date','id']:
            heterodata[nodetype][key] = [more_mappings[key][i.item()] for i in heterodata[nodetype].x]
    
    del heterodata[nodetype].x

paper
author
category
word
journal


In [10]:
import pickle
import os
if not os.path.exists('arxiv_author_paper_graph_no_features_bare.pkl'):
    pickle.dump(heterodata, open("arxiv_author_paper_graph_no_features_bare.pkl", "wb"))

In [11]:
# paper nans to median
import math
non_nan = [num for num in heterodata['paper'].pages if not math.isnan(num)]
median = non_nan[len(non_nan)//2]
heterodata['paper'].pages = [median if math.isnan(num) else num for num in heterodata['paper'].pages]


In [12]:
# categorical 
unique_licenses= list(set(heterodata['paper'].license))
categories = {unique_licenses[i]:i for i in range(len(unique_licenses))}
heterodata['paper'].license = [categories[license] for license in heterodata['paper'].license]
import torch 
# onehot
onehot = torch.nn.functional.one_hot(torch.tensor(heterodata['paper'].license ))
heterodata['paper'].license = onehot

In [13]:
given_timestamp = heterodata['paper'].date[0]
import datetime
start_timestamp_epoch = datetime.datetime(1970, 1, 1, 0, 0, 0, tzinfo=given_timestamp.tzinfo)

# Calculate the difference in hours divided by 
difference_in_hours_by_4 = (given_timestamp - start_timestamp_epoch).total_seconds() // (3600*4) 

In [14]:
heterodata['paper'].date = [((timestamp - start_timestamp_epoch).total_seconds() // (3600*4)) for timestamp in heterodata['paper'].date]

In [15]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('all-MiniLM-L6-v2')
from tqdm.auto import tqdm
# for each node type create the x attribute from the embeddings of the node names
for node_type in heterodata.node_types:
    print(node_type)
    # do it in batches
    x = torch.zeros(heterodata[node_type].num_nodes, 384).float()
    batch_size = 10000
    for i in tqdm(range(0, heterodata[node_type].num_nodes, 10000)):
        x[i:i+10000] = torch.tensor(embedder.encode(heterodata[node_type].name[i:i+10000], convert_to_numpy=True))
        
        
    heterodata[node_type].x = x

paper


100%|██████████| 1/1 [00:03<00:00,  3.62s/it]


author


100%|██████████| 1/1 [00:01<00:00,  1.12s/it]


category


100%|██████████| 1/1 [00:00<00:00, 19.73it/s]


word


100%|██████████| 2/2 [00:03<00:00,  1.89s/it]


journal


100%|██████████| 1/1 [00:00<00:00,  2.64it/s]


In [16]:
heterodata['paper'].x = torch.concatenate((heterodata['paper'].x,heterodata['paper'].license), dim=1)
heterodata['paper'].x  = torch.concatenate((heterodata['paper'].x, torch.tensor(heterodata['paper'].date).unsqueeze(1)), dim=1)

In [17]:
for node_type in heterodata.node_types:
    del heterodata[node_type].name 
    if node_type == 'paper':
        for key in ['license','doi','pages','journal','date','id']:
            del heterodata[node_type][key]

In [ ]:
# save 
import pickle
import os
if not os.path.exists('arxiv_author_paper_graph_training_v1.pkl'):
    pickle.dump(heterodata, open("arxiv_author_paper_graph_training_v1.pkl", "wb"))
